# Assignment 10

Previous assignment parts

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
# importing the external files
ExternalFilesFolder = r"C:\Users\Famiglia\Documents\Manuel\Polimi\MAGISTRALE\PRIMO ANNO\Primo semestre\Bezhad\python4ScientificComputing_Numpy_Pandas_MATPLotLIB\ExternalFiles"
ConsumptionFileName= "consumption_5545.csv"
TemperatureFileName= "Austin_weather_2014.csv"
IrradianceFileName= "irradiance_2014_gen.csv"

path_consumptionFile = os.path.join(ExternalFilesFolder,ConsumptionFileName)
path_TemperatureFile = os.path.join(ExternalFilesFolder,TemperatureFileName)
path_IrradianceFile = os.path.join(ExternalFilesFolder,IrradianceFileName)

DF_consumption = pd.read_csv(path_consumptionFile,sep=",", index_col=0)

PreviousIndex = DF_consumption.index
NewParsedIndex= pd.to_datetime(PreviousIndex)
DF_consumption.index =NewParsedIndex 
DF_weather = pd.read_csv(path_TemperatureFile,sep=";",index_col=0)

previousIndex_weather=DF_weather.index
newIndex_weather=pd.to_datetime(previousIndex_weather)
DF_weather.index = newIndex_weather
DF_Temperature= DF_weather[["temperature"]]

DF_irradianceSource = pd.read_csv(path_IrradianceFile,sep=";",index_col=1)

DF_irradiance=DF_irradianceSource[["gen"]]
DF_irradiance[DF_irradiance["gen"]<0] = 0
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])
DF_joined_cleaned = DF_joined.dropna()
DF_joined = DF_consumption.join([DF_Temperature,DF_irradiance])
DF_mod = DF_joined.copy()

def lag_feature(df,column_name,lag_start,lag_end,lag_interval):
    for i in range(lag_start,lag_end+1,lag_interval):
        new_column_name = column_name+" -"+str(i)+"hr"
        df[new_column_name]=df[column_name].shift(i)   
        df.dropna(inplace=True) #this removes all the row with a Nan
    return df

DF_mod=DF_mod.rename(columns={"air conditioner_5545":"AC_consumption","gen":"Irradiance"})

DF_mod=  lag_feature(DF_mod,"temperature",1,6,1)
DF_mod=  lag_feature(DF_mod,"Irradiance",3,6,1)
DF_mod=  lag_feature(DF_mod,"AC_consumption",1,24,1)
DF_mod.describe()

DF_mod["hour"]=DF_mod.index.hour
DF_mod["sin_hour"]=np.sin(DF_mod.index.hour*2*np.pi/24)
DF_mod["day_of_week"]=DF_mod.index.dayofweek
DF_mod["month"]=DF_mod.index.month
DF_mod["week_of_year"]=DF_mod.index.week

def WeekendDetector(day):
    if (day==5 or day == 6):
        weekendLabel=1
    else:
        weekendLabel=0
    return weekendLabel
DF_mod["weekend"]= DF_mod["day_of_week"].apply(WeekendDetector)
    
def DayDetector(hour):
    if (hour< 19 and hour>=9):
        DayLabel=1
    else:
        DayLabel=0
    return DayLabel
DF_mod["workingTime"] = DF_mod["hour"].apply(DayDetector)

DF_mod.corr()   #to correlate AC consumption wrt all the other values

C:\Users\Famiglia\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Famiglia\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\pandas\core\frame.py:2346: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer(indexer, value)
C:\Users\Famiglia\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\pandas\core\frame.py:2326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas

,AC_consumption,temperature,Irradiance,temperature -1hr,temperature -2hr,temperature -3hr,temperature -4hr,temperature -5hr,temperature -6hr,Irradiance -3hr,...,AC_consumption -22hr,AC_consumption -23hr,AC_consumption -24hr,hour,sin_hour,day_of_week,month,week_of_year,weekend,workingTime
AC_consumption,1.000000,0.630742,-0.012853,0.599176,0.556394,0.505934,0.451338,0.397252,0.347387,0.364446,...,0.769706,0.849283,0.899836,0.360966,-0.438457,-0.015430,0.129079,0.140247,-0.006669,-0.053457
temperature,0.630742,1.000000,-0.118013,0.990939,0.968245,0.935151,0.895070,0.851201,0.806380,0.140999,...,0.535900,0.575420,0.604337,0.177237,-0.227335,0.033468,0.168134,0.198513,0.037580,-0.169694
Irradiance,-0.012853,-0.118013,1.000000,-0.172247,-0.205359,-0.218040,-0.212754,-0.192737,-0.160926,0.560616,...,0.219591,0.091804,-0.025421,0.167031,-0.236701,-0.030422,-0.034063,-0.026283,-0.032257,0.766331
temperature -1hr,0.599176,0.990939,-0.172247,1.000000,0.990913,0.968217,0.935110,0.895008,0.851117,0.043944,...,0.488340,0.535893,0.575415,0.102793,-0.145983,0.033141,0.169105,0.199308,0.037855,-0.232486
temperature -2hr,0.556394,0.968245,-0.205359,0.990913,1.000000,0.990927,0.968234,0.935125,0.895016,-0.044701,...,0.436201,0.488356,0.535913,0.019799,-0.054629,0.032996,0.170006,0.200080,0.038383,-0.279430
temperature -3hr,0.505934,0.935151,-0.218040,0.968217,0.990927,1.000000,0.990926,0.968228,0.935108,-0.118207,...,0.385171,0.436201,0.488359,-0.065501,0.040507,0.032876,0.170881,0.200886,0.039383,-0.306254
temperature -4hr,0.451338,0.895070,-0.212754,0.935110,0.968234,0.990926,1.000000,0.990924,0.968219,-0.172448,...,0.338256,0.385167,0.436199,-0.145732,0.132945,0.032662,0.171714,0.201686,0.040585,-0.313938
temperature -5hr,0.397252,0.851201,-0.192737,0.895008,0.935125,0.968228,0.990924,1.000000,0.990921,-0.205584,...,0.298872,0.338246,0.385161,-0.214423,0.216366,0.032251,0.172486,0.202462,0.041582,-0.300198
temperature -6hr,0.347387,0.806380,-0.160926,0.851117,0.895016,0.935108,0.968219,0.990921,1.000000,-0.218300,...,0.269780,0.298853,0.338230,-0.267137,0.285117,0.031728,0.173177,0.203201,0.042507,-0.268127
Irradiance -3hr,0.364446,0.140999,0.560616,0.043944,-0.044701,-0.118207,-0.172448,-0.205584,-0.218300,1.000000,...,0.509455,0.442437,0.342882,0.465233,-0.691769,-0.030278,-0.034201,-0.026419,-0.032309,0.536805


We are preparing the data: the target is our real values, we want to predict them by different predicting models

In [2]:
DF_mod=DF_mod["2014-03-01":"2014-09-30"]  #air conditioning period


DF_target=DF_mod["AC_consumption"] 
DF_features=DF_mod.drop("AC_consumption",axis=1)

Importing from sklearn a function that splits the data in one part for training the model and one for testing it

In [9]:
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(DF_features,DF_target,test_size = 0.2, random_state=41234)

Importing linear regression model, fitting it with training to get results from model, predicting with X data test, and test the model confronting with Y_test

In [10]:
from sklearn import linear_model
linear_reg = linear_model.LinearRegression()

# The second step will be fitting a model
linear_reg.fit(X_train, Y_train)

predicted_linearReg_split = linear_reg.predict(X_test)

predicted_DF_linearReg_split=pd.DataFrame(predicted_linearReg_split,index=Y_test.index, columns=["AC_cons_predicted_linearReg_split"])
predicted_DF_linearReg_split=predicted_DF_linearReg_split.join(Y_test)

predicted_DF_linearReg_split_august=predicted_DF_linearReg_split["2014-08-01":"2014-08-31"]
predicted_DF_linearReg_split_august.plot()

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

MAE_linearReg_split=mean_absolute_error(predicted_linearReg_split,Y_test)
MSE_linearReg_split=mean_squared_error(predicted_linearReg_split,Y_test)
R2_linearReg_split=r2_score(predicted_linearReg_split,Y_test)

Using linear regression but cutting the data in different way, as k-fold cross validation method, to find better (higher) R2

In [11]:
#second way to find accuration using k-fold cross validation
from sklearn.model_selection import cross_val_predict

predict_linearReg_CV=cross_val_predict(linear_reg,DF_features,DF_target,cv=10)
predicted_DF_linearReg_CV=pd.DataFrame(predict_linearReg_CV,index=DF_target.index,columns=["AC_cons_predicted_linearReg_CV"] )  #to compare Y_test wrt predicted_linearReg_split
predicted_DF_linearReg_CV=predicted_DF_linearReg_CV.join(DF_target) #join to data frame means to match the index!! 

predicted_DF_linearReg_CV_august=predicted_DF_linearReg_CV["2014-08-01":"2014-08-30"]
predicted_DF_linearReg_CV_august.plot()

#accuration of prediction
MAE_linearReg_CV= mean_absolute_error(predict_linearReg_CV,DF_target)
MSE_linearReg_CV= mean_squared_error(predict_linearReg_CV,DF_target)
R2_linearReg_CV = r2_score(predict_linearReg_CV,DF_target)

Other algorithm to improve R2, changing the method, choosing random data values for training

In [12]:
from sklearn.ensemble import RandomForestRegressor
reg_RF=RandomForestRegressor()

predict_RF_CV=cross_val_predict(reg_RF,DF_features,DF_target,cv=10)
predicted_DF_RF_CV=pd.DataFrame(predict_RF_CV,index=DF_target.index,columns=["AC_cons_predicted_linearReg_CV"] )  #to compare Y_test wrt predicted_linearReg_split
predicted_DF_RF_CV=predicted_DF_RF_CV.join(DF_target) #join to data frame means to match the index!! 

predicted_DF_RF_CV_august=predicted_DF_RF_CV["2014-08-01":"2014-08-30"]
predicted_DF_RF_CV_august.plot()

MAE_RF_CV=mean_absolute_error(predict_RF_CV,DF_target)
MSE_RF_CV=mean_squared_error(predict_RF_CV,DF_target)
R2_RF_CV=r2_score(predict_RF_CV,DF_target)

Using online learning: initialing variables for while loop

In [14]:
DF_onlineConsumptionPrediction=pd.DataFrame(index=DF_mod.index)  #empty data frame
period_of_training=pd.Timedelta(30,unit="d")

FirstTimeStamp_measured=DF_mod.index[0]
LastTimeStamp_measured=DF_mod.index[-1]

FirstTimeStamp_toPredict=FirstTimeStamp_measured+period_of_training

training_startTimeStamp=FirstTimeStamp_measured
training_endTimeStamp=FirstTimeStamp_toPredict
timeStamp_toPredict=FirstTimeStamp_toPredict

DF_onlineConsumptionPrediction=DF_onlineConsumptionPrediction.truncate(before=training_endTimeStamp)  #we want to predict after that this period, so we delete the previous period

While-loop to create predicting value with linear regression algorithm, using one month for training and one hour for testing

In [17]:
while(timeStamp_toPredict<LastTimeStamp_measured): 
   
    DF_feature_train=DF_features.truncate(before=training_startTimeStamp,after=training_endTimeStamp)
    DF_target_train=DF_target.truncate(before=training_startTimeStamp,after=training_endTimeStamp)
    
    DF_feature_test=DF_features.loc[timeStamp_toPredict].values.reshape(1,-1)
    DF_target_test=DF_target.loc[timeStamp_toPredict]
    
    linear_reg.fit(DF_feature_train,DF_target_train)  #we're training with my linear regression model
    predicted_Consumption=linear_reg.predict(DF_feature_test)   #prediction of the specific hour ---> only one number
    
    DF_onlineConsumptionPrediction.loc[timeStamp_toPredict,"Predicted"]=predicted_Consumption
    DF_onlineConsumptionPrediction.loc[timeStamp_toPredict,"Real"]=DF_target_test
    #now we enlarge the data training adding the data of the hour we are predicting using the real value data
    timeStamp_toPredict=timeStamp_toPredict+pd.Timedelta(1,"h")
    training_endTimeStamp=training_endTimeStamp+pd.Timedelta(1,"h")
    training_startTimeStamp=training_startTimeStamp+pd.Timedelta(1,"h")
    
DF_onlineConsumptionPrediction.dropna(inplace=True)

R2_score_linearReg=r2_score(DF_onlineConsumptionPrediction[["Real"]],DF_onlineConsumptionPrediction[["Predicted"]])

While-loop to create predicting value with random forest algorithm, using one month for training and one hour for testing

In [ ]:

period_of_training=pd.Timedelta(30,unit="d")

FirstTimeStamp_measured=DF_mod.index[0]
LastTimeStamp_measured=DF_mod.index[-1]

FirstTimeStamp_toPredict=FirstTimeStamp_measured+period_of_training

training_startTimeStamp=FirstTimeStamp_measured
training_endTimeStamp=FirstTimeStamp_toPredict
timeStamp_toPredict=FirstTimeStamp_toPredict

DF_onlineConsumptionPrediction["Predicted_RF"]=0

DF_onlineConsumptionPrediction=DF_onlineConsumptionPrediction.truncate(before=training_endTimeStamp)  #we want to predict after that this period, so we delete the previous period

while(timeStamp_toPredict<LastTimeStamp_measured): #1 month for training and 1 hour for testing
   
    DF_feature_train=DF_features.truncate(before=training_startTimeStamp,after=training_endTimeStamp)
    DF_target_train=DF_target.truncate(before=training_startTimeStamp,after=training_endTimeStamp)
    
    DF_feature_test=DF_features.loc[timeStamp_toPredict].values.reshape(1,-1)
    DF_target_test=DF_target.loc[timeStamp_toPredict]
    
    reg_RF.fit(DF_feature_train,DF_target_train)  #we're training with my RF model
    predicted_Consumption_RF=reg_RF.predict(DF_feature_test)   #prediction of the specific hour ---> only one number
    
    DF_onlineConsumptionPrediction.loc[timeStamp_toPredict,"Predicted_RF"]=predicted_Consumption_RF
    DF_onlineConsumptionPrediction.loc[timeStamp_toPredict,"Real"]=DF_target_test
    
    timeStamp_toPredict=timeStamp_toPredict+pd.Timedelta(1,"h")
    training_endTimeStamp=training_endTimeStamp+pd.Timedelta(1,"h")
    training_startTimeStamp=training_startTimeStamp+pd.Timedelta(1,"h")

DF_onlineConsumptionPrediction.dropna(inplace=True)

R2_score_DF_reg_RF=r2_score(DF_onlineConsumptionPrediction[["Real"]],DF_onlineConsumptionPrediction[["Predicted_RF"]])